References:

[Malayalam Baseline Model](https://kurianbenoy.com/ml-blog/malayalamtextmodels/malayalam/nlp/opensource/ml/deep%20learning/smc/2022/06/04/baselinemodel.html)

[Duplicate your repo!](https://huggingface.co/spaces/osanseviero/repo_duplicator)

[Widgets](https://huggingface.co/docs/hub/models-widgets#enabling-a-widget)

In [4]:
repo_id = "hugginglearners/ml-news-classify-fastai"

# 1. Prerequistes

In [7]:
# Installing from main and specifying ["fastai"] to automatically install the relevant packages and versions
# %%capture
!pip install -Uqq huggingface_hub["fastai"] gradio datasets # ohmeow-blurr 
# not needed just for reference:
#!pip install -Uqq transformers transformers['sentencepiece'] datasets

     |████████████████████████████████| 86 kB 4.1 MB/s 
     |████████████████████████████████| 362 kB 38.2 MB/s 
     |████████████████████████████████| 81 kB 10.6 MB/s 
     |████████████████████████████████| 5.1 MB 61.9 MB/s 
     |████████████████████████████████| 1.1 MB 47.0 MB/s 
     |████████████████████████████████| 256 kB 76.4 MB/s 
     |████████████████████████████████| 2.0 MB 52.0 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 140 kB 70.8 MB/s 
     |████████████████████████████████| 84 kB 4.3 MB/s 
     |████████████████████████████████| 53 kB 2.7 MB/s 
     |████████████████████████████████| 212 kB 73.8 MB/s 
     |████████████████████████████████| 212 kB 74.5 MB/s 
     |████████████████████████████████| 596 kB 57.5 MB/s 
     |████████████████████████████████| 127 kB 73.3 MB/s 
     |████████████████████████████████| 4.4 MB 56.3 MB/s 
     |████████████████████████████████| 43 kB 2.7 MB/s 
     |███████████████████

In [6]:
%%capture
!git clone https://github.com/ohmeow/blurr.git
%cd blurr
!pip install -e ".[dev]"

In [4]:
import torch
import datasets
import transformers
from fastai.text.all import *

from blurr.text.data.all import *
from blurr.text.modeling.all import *

# 2. Prep Data

In [5]:
raw_datasets = datasets.load_dataset("hugginglearners/malayalam_news")#rajeshradhakrishnan/malayalam_news

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset malayalam_news downloaded and prepared to /root/.cache/huggingface/datasets/hugginglearners___malayalam_news/malayalam_news/1.0.0/7d6fb88a050b410d95b4f750165a6f797581be76cc65793eb4c6b71ab9ca9b09. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
raw_datasets['train'].features["label"].names

['business', 'entertainment', 'sports', 'technology']

In [7]:
def addTargeCols(example):
    return {"business": 1 if example["label"] == 0 else 0,
            "entertainment": 1 if example["label"] == 1 else 0,
            "sports": 1 if example["label"] == 2 else 0,
            "technology": 1 if example["label"] == 3 else 0}

In [8]:
raw_datasets = raw_datasets.map(addTargeCols)

Parameter 'function'=<function addTargeCols at 0x7f5663d5a7a0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/4800 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

In [9]:
raw_datasets['train'] = raw_datasets['train'].add_column("is_valid", [False] * len(raw_datasets['train']))
raw_datasets['test'] = raw_datasets['test'].add_column("is_valid", [True] * len(raw_datasets['test']))

In [10]:
raw_datasets['train'][:2]

{'business': [0, 0],
 'entertainment': [0, 0],
 'is_valid': [False, False],
 'label': [2, 2],
 'sports': [1, 1],
 'technology': [0, 0],
 'text': ['മത്സര പ്രതിഫലമായി സ്വന്തമാക്കിയത് പതിനേഴ് ദശലക്ഷം ഡോളർ . ബാക്കി തുകയത്രയും പരസ്യ വരുമാനത്തിലൂടെ സ്വന്തമാക്കിയതാണ് . ബോൾട്ടിനോളം വിപണിമൂല്യമുള്ള മറ്റൊരു അത്ലറ്റില്ല . ഇതുകൊണ്ടു തന്നെയാണ് ബോൾട്ട് ലോകത്തെ ഏറ്റവും വരുമാനമുള്ള 100 കായിക താരങ്ങളുടെ പട്ടികയിലെ ഏക അത്ലറ്റായത് . പട്ടികയിൽ ഇരുപത്തിമൂന്നാം സ്ഥാനത്താണ് ജമൈക്കൻ ഇതിഹാസം . വിരമിച്ചതിന് ശേഷം ഫുട്ബോളിൽ സജീവമാകാൻ ഒരുങ്ങുന്ന വേഗരാജാവ് ഉസൈൻ ബോൾട്ട് ഫൌണ്ടേഷനും തുടക്കമിട്ടുകഴിഞ്ഞു . പാവപ്പെട്ട കുട്ടികളുടെ വിദ്യാഭ്യാസത്തിനാണ് ഫൌണ്ടേഷൻ പ്രധാനമായും സഹായിക്കുക . എട്ട് ഒളിംപിക് സ്വർണത്തിന് ഉടമയായ ബോൾട്ടിന്റെ പേരിലാണ് 100,200 മീറ്ററുകളിലെ ലോകറെക്കോർഡ് .',
  'ഇന്ത്യൻ പ്രീമിയർ ലീഗിൽ കൊൽക്കത്ത നൈറ്റ് റൈഡേഴ്സിനെതിരേ സൺറൈസേഴ്സിന് 140 റൺസ് വിജയലക്ഷ്യം . ടോസ് നഷ്ടപ്പെട്ട് ആദ്യ ബാറ്റിങ് ആരംഭിച്ച കൊൽത്തക്കയ്ക്ക് എട്ട് വിക്കറ്റ് നഷ്ടത്തിൽ 139 റൺസ് മാത്രമാണ് എടുക്കാൻ സാധിച്ചത് . 49 റൺസെടുത്ത ക്രിസ് ലിന്നാണ് അവരുട

In [11]:
df = raw_datasets.filter(lambda x: x["entertainment"] == 0)
print(df.num_rows)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

{'train': 3578, 'validation': 455, 'test': 467}


In [12]:
final_ds = datasets.concatenate_datasets([raw_datasets['train'].shuffle().select(range(1000))
                                        , raw_datasets['test'].shuffle().select(range(200))])
mlnews_df = pd.DataFrame(final_ds)
mlnews_df.head()

,text,label,business,entertainment,sports,technology,is_valid
0,ഉപഗ്രഹങ്ങളുടെ കൂട്ട വിക്ഷേപണത്തിലൂടെ കഴിഞ്ഞ സാമ്പത്തിക വർഷം വൻ നേട്ടം കൊയ്ത് ഐഎസ്ആർഒ . ഐഎസ്ആർഒയുടെ വിശ്വസ്ത ലോഞ്ചിംഗ് വെഹിക്കിളായ പിഎസ്എൽവി ഇതുവരെ 28 രാജ്യങ്ങളിൽ നിന്നുള്ള 209 ഉപഗ്രഹങ്ങളാണ് ബഹിരാകാശത്ത് എത്തിച്ചത് . ഇതുവഴി ഐഎസ്ആർ വൻ സാമ്പത്തിക നേട്ടമുണ്ടാക്കിയതായി കണക്കുകൾ പറയുന്നു . ഐഎസ്ആർഒയുടെ വാണിജ്യ വിഭാഗമായ ആൻട്രിക്സിലൂടെ വിദേശ രാജ്യങ്ങളുടെ വാണിജ്യ ഉപഗ്രഹങ്ങൾ വിക്ഷേപിച്ച് 2015 - 2016 സാമ്പത്തികവർഷം 230 കോടി രൂപയാണ് ഐഎസ്ആർഒ സ്വന്തമാക്കിയത് . 2013 - മുതൽ 2015 വരെയുള്ള രണ്ടുവർഷ കാലയളവിൽ 28 വിദേശ ഉപഗ്രഹങ്ങൾ വിക്ഷേപിച്ചതുവഴി ആൻട്രിക്സ് 600 കോടി രൂപയുടെ ആകെ നേട്ടമുണ്ടാക്കിയിരുന്നു . ഈ വർഷം ...,3,0,0,0,1,False
1,വിവിധ തിയ്യേറ്ററുകളിൽ മോഹൻലാൽ ചിത്രം ഒടിയന്റെ ആദ്യ പ്രദർശനം തുടങ്ങി . ഹർത്താലിന്റെ പശ്ചാത്തലത്തിൽ ആഘോഷങ്ങൾ കുറവാണെങ്കിലും തിയ്യേറ്റർ ഹൌസ്ഫുള്ളാണ് . 37 രാജ്യങ്ങളിലായി 3500 കേന്ദ്രങ്ങളിലായിരുന്ന റിലീസ് . എന്നാൽ ഹർത്താലിന്റെ പശ്ചാത്തലത്തിൽ ചില തിയ്യേറ്ററുകളിലെ ഷോ മാറ്റിവച്ചു . സുരക്ഷാ പ്രശ്നമുള്ളതിനാൽ വൈകീട്ട് ആറ് മണിക്ക് ശേഷം ചിത്രം പ്രദർശിപ്പിക്കും . ചിലയിടങ്ങളിൽ പ്രദർശനം നിർത്തിവച്ചിട്ടുണ്ട് . പ്രദർശനം നടക്കാത്തതിലുള്ള അമർശവും ചില ആരാധകർ ഏഷ്യാനെറ്റ് ന്യൂസിനോട് പങ്കുവച്ചു . ഇന്നലെ രാത്രി വരെ അന്വേഷിച്ചപ്പോൾ ഷോ പ്രദർശനം നടക്കുമെന്ന് അറിയിച്ചിട്ടാണ് എത്തിയതെന്നും ആരാധകർ പറഞ്ഞു . എന്നാൽ പൊലീസ്...,1,0,1,0,0,False
2,"ഇന്ത്യക്കെതിരായ ഏകദിന പരമ്പരയ്ക്കുള്ള ടീമിനെ ഇന്ന് രാവിലെയാണ് ക്രിക്കറ്റ് ഓസ്ട്രേലിയ പ്രഖ്യാപിച്ചത് . പേസർ പീറ്റർ സിഡിൽ , സ്പിന്നർ നഥാൻ ലിയോൺ , മുൻനിര ബാറ്റ്സ്മാൻ ഉസ്മാൻ ഖവാജ എന്നിവർ ടീമിൽ തിരിച്ചെത്തിയപ്പോൾ മിച്ചൽ സ്റ്റാർക്ക് , ജോഷ് ഹേസൽവുഡ് , പാറ്റ് കമ്മിൺസ് എന്നിവർക്ക് വിശ്രമം അനുവദിച്ചിരുന്നു . ഇന്ത്യയെ നേരിടാൻ ഈ ടീം മതിയോ എന്ന സംശയം ടീം പ്രഖ്യാപനത്തിന് പിന്നാലെ വിവിധ കോണുകളിൽ ഉയർന്നിരുന്നു . ഇപ്പോഴിതാ ക്രിക്കറ്റ് ഓസ്ട്രേലിയയുടെ ടീം സെലക്ഷനെതിരെ പരസ്യമായി പ്രതികരിച്ച് ഇതിഹാസ താരം ഷെയ്ൻ വോൺ രംഗത്തെത്തി . ലോകകപ്പ് പോരാട്ടം പടിവാതിൽക്കൽ എത്തി നിൽക്കെയുള്ള ക്രിക്കറ്റ് ഓസ്ട്രേലിയയുടെ ടീം സെ...",2,0,0,1,0,False
3,"സംസ്കൃതത്തിൽ ഗജമെന്നാൽ ആനയെന്നാണ് അർഥം . തമിഴ്നാട്ടിൽ അതിരാമപട്ടണത്തിന് പടിഞ്ഞാറ് മണിക്കൂറിൽ 95 കിലോമീറ്റർ വേഗത്തിൽ വീശിയടിച്ച ചുഴലിക്കാറ്റിന് പേരിട്ടത് ശ്രീലങ്കയാണ് . ആനയുടെ ശക്തിയുള്ള കാറ്റ് എന്ന അർഥത്തിൽ . ഗജ വെള്ളിയാഴ്ച മൂന്നു മണിയോടെ ന്യൂനമർദമായി മാറി കേരള തീരത്തിലെത്തി ഇടുക്കി , പാലക്കാട് ജില്ലകളിലൂടെ കേരളം കടന്ന് അറബിക്കടലിലേക്ക് നീങ്ങി . വെള്ളിയാഴ്ചയും ശനിയാഴ്ചയും ചില ജില്ലകളിൽ ഇതിന്റെ ഭാഗമായി ശക്തമായ മഴ ഉണ്ടാകുമെന്ന് കാലാവസ്ഥാ നിരീക്ഷണ കേന്ദ്രം പറയുന്നു . ഒഡീഷ തീരത്ത് വീശിയ തിത്ലി ചുഴലിക്കാറ്റിനുശേഷം ഗജ എത്തുമ്പോൾ പേരുകളിലെ വ്യത്യസ്തതയാണ് ആദ്യം ശ്രദ്ധിക്കപ്പെടുന്നത് . ചുഴലിക്കാറ്റ...",3,0,0,0,1,False
4,"ആപ്പിൾ ഡ്രൈവർ ആവശ്യമില്ലാത്ത കാറുകൾ നിർമ്മിക്കാൻ ഒരുങ്ങുന്നു . ആപ്പിൾ മേധാവി ടിം കുക്ക് തന്നെയാണ് ഈ കാര്യം വ്യക്തമാക്കിയത് . ബ്ലൂബെർഗിന് നൽകിയ അഭിമുഖത്തിലാണ് ടെക് ഭീമൻറെ വെളിപ്പെടുത്തൽ . ജൂൺ 5 ന് പ്രസിദ്ധീകരിച്ച അഭിമുഖത്തിൽ ഓട്ടോണമസ് ഓട്ടോമൊബൈൽ രംഗത്തേക്ക് ആപ്പിൾ ഉടൻ കടന്നുവരും എന്നാണ് ടിംകുക്ക് പറയുന്നത് . എങ്ങനെയിരിക്കും പ്രോഡക്ട് എന്നോ , പദ്ധതി എങ്ങനെയിരിക്കും എന്നോ ഇപ്പോൾ വ്യക്തമാക്കുവാൻ സാധിക്കില്ല . എന്നാൽ ഞങ്ങൾ അത് ചെയ്തിരിക്കും . ഇത് ഒരു കേന്ദ്രീകൃതമായ സാങ്കേതികതയിൽ അടിസ്ഥിതമായിരിക്കുമെന്നും ടിം കുക്ക് പുതിയ ടെക്നോളജിയെക്കുറിച്ച് സൂചന നൽകുന്നു . കഴിഞ്ഞ ഏപ്രിലിൽ ആപ്പിൾ തങ്ങളുടെ ഓട്ട...",3,0,0,0,1,False


# 3. Using Blurr High Level API

In [13]:
lbl_cols = raw_datasets['train'].features["label"].names
lbl_cols

['business', 'entertainment', 'sports', 'technology']

In [14]:
from transformers import AutoConfig, AutoModelForSequenceClassification
model_cls = AutoModelForSequenceClassification

pretrained_model_name = "xlm-roberta-base"
config = AutoConfig.from_pretrained(pretrained_model_name)
config.num_labels = len(lbl_cols)

hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(pretrained_model_name, model_cls=model_cls, config=config)
hf_model.config.problem_type = "multi_label_classification"

print(hf_arch)
print(type(hf_config))
print(type(hf_tokenizer))
print(type(hf_model))

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

xlm_roberta
<class 'transformers.models.xlm_roberta.configuration_xlm_roberta.XLMRobertaConfig'>
<class 'transformers.models.xlm_roberta.tokenization_xlm_roberta_fast.XLMRobertaTokenizerFast'>
<class 'transformers.models.xlm_roberta.modeling_xlm_roberta.XLMRobertaForSequenceClassification'>


In [15]:
learn = BlearnerForSequenceClassification.from_data(
    mlnews_df, pretrained_model_name, text_attr="text", label_attr=lbl_cols, dl_kwargs={"bs": 4}
)

learn.loss_func.thresh = 0.02

In [16]:
learn.fit_one_cycle(1, lr_max=1e-2)

epoch,train_loss,valid_loss,f1_score,accuracy_multi,time
0,0.595574,0.570942,0.000000,0.750000,00:48


In [17]:
learn.loss_func.thresh = 0.01

comment = """
2032 ഒളിമ്പിക്‌സിന് ബ്രിസ്‌ബെയ്ന്‍ വേദിയാകും; ഗെയിംസിന് വേദിയാകുന്ന മൂന്നാമത്തെ ഓസ്‌ട്രേലിയന്‍ നഗരം
"""
learn.predict(comment)

[{'class_indices': [1, 1, 1, 1],
  'class_labels': ['business', 'entertainment', 'sports', 'technology'],
  'labels': ['business', 'entertainment', 'sports', 'technology'],
  'probs': [0.3053056001663208,
   0.1699194461107254,
   0.2217995971441269,
   0.26829949021339417],
  'scores': [0.3053056001663208,
   0.1699194461107254,
   0.2217995971441269,
   0.26829949021339417]}]

# 5. Share in Huggingface

In [18]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [19]:
from huggingface_hub import push_to_hub_fastai

push_to_hub_fastai(learner=learn, repo_id=repo_id)

Cloning https://huggingface.co/hugginglearners/ml-news-classify-fastai into local empty directory.
Adding files tracked by Git LFS: ['model.pkl']. This may take a bit of time if the files are large.


Upload file model.pkl:   0%|          | 3.33k/1.04G [00:00<?, ?B/s]

To https://huggingface.co/hugginglearners/ml-news-classify-fastai
   269e318..28ab382  main -> main



'https://huggingface.co/hugginglearners/ml-news-classify-fastai/commit/28ab382cf93d199f434c5445221dfad2b8f3172a'

# 6. Loading a Learner from the Hugging Face Hub

In [13]:
from huggingface_hub import from_pretrained_fastai
# ഞാൻ ചിന്തിക്കുന്ന കുറച്ചു ആശയങ്ങൾ പ്രാവർത്തികം ആക്കുക എന്നുള്ളതാണ്
# ഇതുവരെ ഞാൻ കണ്ടുമുട്ടിയതും പരിചയവുമുള്ള സോഫ്റ്റ്‌വെയർ സേവനങ്ങൾ 
# എങ്ങനെ പ്രയോജനപ്പെടുത്താം എന്നുള്ള വിവരങ്ങൾ ഇവിടെ പരിചയ പെടുത്തുന്നതുമാണ്. 
learner = from_pretrained_fastai(repo_id)

Downloading:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [15]:
sentences = ["ഓഹരി വിപണി തകരുമ്പോള്‍ നിക്ഷേപം എങ്ങനെ സുരക്ഷിതമാക്കാം",
             "വാര്‍ണറുടെ ഒറ്റക്കയ്യന്‍ ക്യാച്ചില്‍ അമ്പരന്ന് ക്രിക്കറ്റ് ലോകം"]

probs = learner.predict(sentences)
# 'business', 'entertainment', 'sports', 'technology'
for idx in range(len(sentences)):
  print(f"Probability that sentence '{sentences[idx]}' is business is: {100*probs[idx]['probs'][0]:.2f}%")
  print(f"Probability that sentence '{sentences[idx]}' is entertainment is: {100*probs[idx]['probs'][1]:.2f}%")
  print(f"Probability that sentence '{sentences[idx]}' is sports is: {100*probs[idx]['probs'][2]:.2f}%")
  print(f"Probability that sentence '{sentences[idx]}' is technology is: {100*probs[idx]['probs'][3]:.2f}%")

Probability that sentence 'ഓഹരി വിപണി തകരുമ്പോള്‍ നിക്ഷേപം എങ്ങനെ സുരക്ഷിതമാക്കാം' is business is: 30.53%
Probability that sentence 'ഓഹരി വിപണി തകരുമ്പോള്‍ നിക്ഷേപം എങ്ങനെ സുരക്ഷിതമാക്കാം' is entertainment is: 16.99%
Probability that sentence 'ഓഹരി വിപണി തകരുമ്പോള്‍ നിക്ഷേപം എങ്ങനെ സുരക്ഷിതമാക്കാം' is sports is: 22.18%
Probability that sentence 'ഓഹരി വിപണി തകരുമ്പോള്‍ നിക്ഷേപം എങ്ങനെ സുരക്ഷിതമാക്കാം' is technology is: 26.83%
Probability that sentence 'വാര്‍ണറുടെ ഒറ്റക്കയ്യന്‍ ക്യാച്ചില്‍ അമ്പരന്ന് ക്രിക്കറ്റ് ലോകം' is business is: 30.53%
Probability that sentence 'വാര്‍ണറുടെ ഒറ്റക്കയ്യന്‍ ക്യാച്ചില്‍ അമ്പരന്ന് ക്രിക്കറ്റ് ലോകം' is entertainment is: 16.99%
Probability that sentence 'വാര്‍ണറുടെ ഒറ്റക്കയ്യന്‍ ക്യാച്ചില്‍ അമ്പരന്ന് ക്രിക്കറ്റ് ലോകം' is sports is: 22.18%
Probability that sentence 'വാര്‍ണറുടെ ഒറ്റക്കയ്യന്‍ ക്യാച്ചില്‍ അമ്പരന്ന് ക്രിക്കറ്റ് ലോകം' is technology is: 26.83%


# 7. Create a Gradio demo to showcase your model

In [1]:
!pip install -Uqq huggingface_hub["fastai"] gradio

     |████████████████████████████████| 86 kB 4.1 MB/s 
     |████████████████████████████████| 5.1 MB 57.7 MB/s 
     |████████████████████████████████| 2.0 MB 44.2 MB/s 
     |████████████████████████████████| 1.1 MB 50.5 MB/s 
     |████████████████████████████████| 53 kB 749 kB/s 
     |████████████████████████████████| 84 kB 3.3 MB/s 
     |████████████████████████████████| 140 kB 55.0 MB/s 
     |████████████████████████████████| 212 kB 34.0 MB/s 
     |████████████████████████████████| 256 kB 57.1 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 94 kB 2.3 MB/s 
     |████████████████████████████████| 271 kB 63.1 MB/s 
     |████████████████████████████████| 144 kB 9.0 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 80 kB 9.8 MB/s 
     |████████████████████████████████| 596 kB 6.6 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |██████████████████████████

In [7]:
import gradio as gr
from huggingface_hub import from_pretrained_fastai

LABELS = ['business', 'entertainment', 'sports', 'technology']
def predict(news_headline):
    learner = from_pretrained_fastai(repo_id)

    probabilities = learner.predict(news_headline)

    return {LABELS[i]: probabilities[0]['probs'][i] for i in range(len(LABELS))}

interface = gr.Interface(
    predict, 
    inputs="textbox", 
    outputs='label', 
    theme="huggingface", 
    title="Malayalam News Classifier", 
    description="Try to classify news in മലയാളം? Input a few malayalam news headlines and verify whether the model categorized it appropriately!", 
    article = "<p style='text-align: center'>Malayalam News Classifier | Demo Model</p>",
    examples=[["ഓഹരി വിപണി തകരുമ്പോള്‍ നിക്ഷേപം എങ്ങനെ സുരക്ഷിതമാക്കാം"], ["വാര്‍ണറുടെ ഒറ്റക്കയ്യന്‍ ക്യാച്ചില്‍ അമ്പരന്ന് ക്രിക്കറ്റ് ലോകം"]],
    # live=True,
    # share=True
    )
interface.launch(debug=True)


/usr/local/lib/python3.7/dist-packages/gradio/interface.py:286: UserWarning: Currently, only the 'default' theme is supported.
  warnings.warn("Currently, only the 'default' theme is supported.")


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://45332.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7f890fdcfa50>,
 'http://127.0.0.1:7860/',
 'https://45332.gradio.app')

In [9]:
# "spaces/rajeshradhakrishnan/malayalam-news-classify"
gr.Interface.load("spaces/hugginglearners/malayalam-news-classify").launch(debug=True) 

Fetching interface from: https://huggingface.co/spaces/hugginglearners/malayalam-news-classify


TypeError: ignored